In [0]:
%load_ext autoreload
%autoreload 2


In [0]:
# carregamento de bibliotecas
import sys
import os
import requests
from datetime import date, timedelta
from io import StringIO
import pandas as pd
root = os.path.abspath(os.path.join(os.getcwd(), "../../"))

if root not in sys.path:
    sys.path.append(root)


from src.features.ingestion import cotacao_dolar_ptax
from src.features.data import criar_tabela_spark, salvar_tabela_delta_spark
from src.features.processing import renomear_colunas, data_ingestao




In [0]:
# VARIÁVEIS DE DESTINO
TABELA_INCREMENTAL_RAW = "`api-forecasting`.bronze.dolar_ptax_incremental_raw"

In [0]:
# construção da tabela histórica de cotação do dolar ptax
API_BC_URL = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/"

# intervalo de data
DATA_INICIAL = (date.today() - timedelta(days=1)).strftime("%m-%d-%Y")
DATA_FINAL = (date.today() - timedelta(days=1)).strftime("%m-%d-%Y")


df_cotacao_dolar = cotacao_dolar_ptax(API_BC_URL, 
                                 DATA_INICIAL, 
                                 DATA_FINAL)

df_cotacao_dolar.head()

In [0]:
# sanitização da tabela para persistência
cols_sanitizacao = {'dataHoraCotacao': 'Data'}

df_cotacao_dolar = renomear_colunas(df = df_cotacao_dolar, 
                                    dicionario = cols_sanitizacao) 
 

df_cotacao_dolar['Data'] = pd.to_datetime(df_cotacao_dolar['Data'], format='mixed')
df_cotacao_dolar.head()


In [0]:
# inserção da data de ingestao
df_cotacao_dolar = data_ingestao(df_cotacao_dolar)
df_cotacao_dolar.head()

In [0]:
# conversão do dataframe no formato spark
df_cotacao_dolar = criar_tabela_spark(df_cotacao_dolar)

In [0]:
# persistência da tabela na camada bronze
salvar_tabela_delta_spark(df=df_cotacao_dolar, 
                          mode='append',
                          merge_schema=True,
                          partitionby=['Data'], 
                          path=TABELA_INCREMENTAL_RAW)

In [0]:
%sql
-- TABELA CATALOGADA NA CAMADA BRONZE
SELECT * 
FROM `api-forecasting`.bronze.dolar_ptax_incremental_raw
LIMIT 10